# UDACITY Demo - Triggering Lambda

In [3]:
%%writefile lambda_function.py

# This cell will write the function to your local machine. Note the name of the file and the name of the function. 
# Compare this to the 'Handler' parameter. 

import json

def lambda_handler(event, context):
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

Overwriting lambda_function.py


In [5]:
import boto3
from sagemaker import get_execution_role ## The SageMaker role executing your notebook needs to have Lambda permissions. 
import json
from zipfile import ZipFile

client = boto3.client('lambda')

with ZipFile('code.zip', 'w') as f:
    f.write('lambda_function.py')
    
# If submitting as a ZipFile, you need to insert raw data. 
    
with open('code.zip', 'rb') as f:
    b_code = f.read()

    
# You will need to submit an 'execution role' to Lambda. The easiest way to add this is through the IAM console. 
# You'll need the full ARN (not just the name. )
# Lambda function names also need to be unique within your AWS account. 

response = client.create_function(
    FunctionName='example123',
    Runtime='python3.9',
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': b_code,
    },
    Description='string',
    Timeout=30,
    MemorySize=1024,
    Publish=True,
    PackageType='Zip',
    Role= 'arn:aws:iam::298735464366:role/service-role/my-first-function-abcdef-role-zn4w17tv' #'arn:aws:iam::565094796913:role/lambda_full_access'#'arn:aws:iam::298735464366:role/AWSLambda_FullAccess'
)

In [6]:
print(response)

{'ResponseMetadata': {'RequestId': 'e8153ed8-990e-4e97-ae8d-3b50e2cb4c0e', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Mon, 19 Sep 2022 08:09:38 GMT', 'content-type': 'application/json', 'content-length': '1014', 'connection': 'keep-alive', 'x-amzn-requestid': 'e8153ed8-990e-4e97-ae8d-3b50e2cb4c0e'}, 'RetryAttempts': 0}, 'FunctionName': 'example123', 'FunctionArn': 'arn:aws:lambda:us-east-1:298735464366:function:example123', 'Runtime': 'python3.9', 'Role': 'arn:aws:iam::298735464366:role/service-role/my-first-function-abcdef-role-zn4w17tv', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 440, 'Description': 'string', 'Timeout': 30, 'MemorySize': 1024, 'LastModified': '2022-09-19T08:09:37.786+0000', 'CodeSha256': '5MMIDIv3hCfl6qCWa8AHhHZcRxZnOe+MKMY4cZORAuY=', 'Version': '1', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '35172953-bd30-442e-aa8d-905be9065cf2', 'State': 'Pending', 'StateReason': 'The function is being created.', 'StateReasonCode': 'Creating', 'Pack

In [7]:


payload = {'key': 'value'}

# json.dumps turns a JSON-object-like python object into a string, and .encode('utf-8') encodes the 
# the string so that it can be properly passed to the client. 

payload_bytes = json.dumps(payload).encode('utf-8')

response = client.invoke(
    FunctionName='example123',
    InvocationType='Event',
    Payload=payload_bytes
)

In [8]:
print(response)# Function was invoked successfully

{'ResponseMetadata': {'RequestId': 'f21c4c0b-acbd-4219-a5fe-89d8ce28a267', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 19 Sep 2022 08:18:19 GMT', 'content-length': '0', 'connection': 'keep-alive', 'x-amzn-requestid': 'f21c4c0b-acbd-4219-a5fe-89d8ce28a267', 'x-amzn-remapped-content-length': '0', 'x-amzn-trace-id': 'root=1-632825cb-3d7a2fcf66e9ef4950d19612;sampled=0'}, 'RetryAttempts': 1}, 'StatusCode': 202, 'Payload': <botocore.response.StreamingBody object at 0x7fd3bd4f4a90>}
